# RAG 기본 구조 이해하기

## 1. 사전작업(Pre-processing) - 1~4 단계

![rag-1.png](./assets/rag-1.png)

![rag-1-graphic](./assets/rag-graphic-1.png)

사전 작업 단계에서는 데이터 소스를 Vector DB (저장소) 에 문서를 로드-분할-임베딩-저장 하는 4단계를 진행합니다.

- 1단계 문서로드(Document Load): 문서 내용을 불러옵니다.
- 2단계 분할(Text Split): 문서를 특정 기준(Chunk) 으로 분할합니다.
- 3단계 임베딩(Embedding): 분할된(Chunk) 를 임베딩하여 저장합니다.
- 4단계 벡터DB 저장: 임베딩된 Chunk 를 DB에 저장합니다.

## 2. RAG 수행(RunTime) - 5~8 단계

![rag-2.png](./assets/rag-2.png)

![](./assets/rag-graphic-2.png)

- 5단계 검색기(Retriever): 쿼리(Query) 를 바탕으로 DB에서 검색하여 결과를 가져오기 위하여 리트리버를 정의합니다. 리트리버는 검색 알고리즘이며(Dense, Sparse) 리트리버로 나뉘게 됩니다. Dense: 유사도 기반 검색, Sparse: 키워드 기반 검색
- 6단계 프롬프트: RAG 를 수행하기 위한 프롬프트를 생성합니다. 프롬프트의 context 에는 문서에서 검색된 내용이 입력됩니다. 프롬프트 엔지니어링을 통하여 답변의 형식을 지정할 수 있습니다.
- 7단계 LLM: 모델을 정의합니다.(GPT-3.5, GPT-4, Claude, etc..)
- 8단계 Chain: 프롬프트 - LLM - 출력 에 이르는 체인을 생성합니다.

## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

_실습을 위해 다운로드 받은 파일을 `data` 폴더로 복사해 주시기 바랍니다_


## 환경설정


API KEY 를 설정합니다.


In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

LangChain으로 구축한 애플리케이션은 여러 단계에 걸쳐 LLM 호출을 여러 번 사용하게 됩니다. 이러한 애플리케이션이 점점 더 복잡해짐에 따라, 체인이나 에이전트 내부에서 정확히 무슨 일이 일어나고 있는지 조사할 수 있는 능력이 매우 중요해집니다. 이를 위한 최선의 방법은 [LangSmith](https://smith.langchain.com)를 사용하는 것입니다.

LangSmith가 필수는 아니지만, 유용합니다. LangSmith를 사용하고 싶다면, 위의 링크에서 가입한 후, 로깅 추적을 시작하기 위해 환경 변수를 설정해야 합니다.


In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH12-RAG")

LangChain/LangSmith API Key가 설정되지 않았습니다. 참고: https://wikidocs.net/250954


## RAG 기본 파이프라인(1~8단계)


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

import os  # For environment variables
import json  # For JSON operations
import chromadb  # For vector database operations
from chromadb.utils import embedding_functions  # For creating embedding functions


아래는 기본적인 RAG 구조 이해를 위한 뼈대코드(skeleton code) 입니다.

각 단계별 모듈의 내용을 앞으로 상황에 맞게 변경하면서 문서에 적합한 구조를 찾아갈 수 있습니다.

(각 단계별로 다양한 옵션을 설정하거나 새로운 기법을 적용할 수 있습니다.)

In [11]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/HuckFinn.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 303


In [12]:
print(docs[10].page_content)

easy in its grave, and has to go about that way every night grieving. I
got so down-hearted and scared I did wish I had some company.
Pretty soon a spider went crawling up my shoulder, and I ﬂipped it
off and it lit in the candle; and before I could budge it was all shriv-
eled up. I didn’t need anybody to tell me that that was an awful bad
sign and would fetch me some bad luck, so I was scared and most
shook the clothes off of me. I got up and turned around in my tracks
three times and crossed my breast every time; and then I tied up a lit-
tle lock of my hair with a thread to keep witches away. But I hadn’t
no conﬁdence.  You do that when you’ve lost a horseshoe that you’ve
found, instead of nailing it up over the door, but I hadn’t ever heard
anybody say it was any way to keep off bad luck when you’d killed a
spider.
I set down again, a-shaking all over, and got out my pipe for a
smoke; for the house was all as still as death now, and so the widow
wouldn’t know. Well, after a long t

`metadata` 를 확인합니다.

In [32]:

# 단계 2.1: 문서 분할(Split Documents)

chunk_size_n = 4000
chunk_overlap_n = 800

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", ","],
    chunk_size=chunk_size_n,
    chunk_overlap=chunk_overlap_n)
split_documents = text_splitter.split_documents(docs)

# 단계 2.2: 문서 분할(Split Documents) using semanticchunking

# text_splitter = SemanticChunker(OpenAIEmbeddings(model = "text-embedding-3-large"))
# split_documents = text_splitter.split_documents(docs) 



# Step 3: Embedding Gen

embeddings = OpenAIEmbeddings(model = "text-embedding-3-large")
# embedded_query = embeddings.embed_query('What was the name mentioned in the coversation?')
# len(embedded_query)


# Step 4: Create Vectorstore + save DB
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(
    documents=split_documents,
    embedding=embeddings)


# Step 5: Creating retriever
# "Retriever" searches and gens based on the doc
retriever = vectorstore.as_retriever()
# testing the retriever and see which chunk was pulled.
retriever.invoke("어린왕자가 네번째 별에서 만난사람은 누구야?")


# Step 6: Setting up the prompt
prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 책(context) 에서 주어진 질문(question) 에 답하는 것입니다.
우선적으로 검색된 다음 문맥(context) 만을 사용하여 질문(question) 에 답하세요. 문서에 직접적인 설명이없더라도, 문맥상으로 유추를 해보고 생각을 곁들여도 괜찮아. 그리고도 답을 모른다면 '잘 모르겠습니다' 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

그리고, 당신은 당신의 지능에 대한 신뢰성을 보여주기위해, 문맥에 의지하는것처럼 보이는건 피해야합니다. '문맥상...'으로 문장을 시작하면 당신이 당신의 대답에 대해 책임을 회피하려는것처럼 보이므로 이를 피해야합니다.

#Context:
{context}

#Question:
{question}

#Answer:"""
)


# Step 7: Create LLM model
llm = ChatOpenAI(model_name="gpt-4.1-2025-04-14", temperature=0)


# Step 8: Create Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## chunk version. 2000, 300. embedding model = "text-embedding-3-small"

In [19]:
## chunk version. 2000, 400. embedding model = "text-embedding-3-large"

# Run Chain
print("hard chunk ver. small embedding model")
print(f"chunk size: {chunk_size_n}")
print(f"overlap size: {chunk_overlap_n}")
print(f"number of chunks: {len(split_documents)}")
print(f"number saved in vecstoredb: {vectorstore.index.ntotal}")


question1 = "주요 등장인물들을 알려줘"
response1 = chain.invoke(question1)
print(f"\nQuestion1:\n{question1}\n", 
      f"\nResponse1:\n{response1}")

question2 = "한문단으로 내용요약을 해줘. 책을 읽어보지않은사람에게 흥미를 일으킬수있도록 아주 흥미진진하게 내용을 소개하는 느낌으로 써줘"
response2 = chain.invoke(question2)
print(f"\nQuestion2:\n{question2}\n",
      f"\nResponse2:\n{response2}")

question3 = "세문단으로 내용요약을 해줘. 기술적으로 사실에 근거해서 책의 모든 내용을 정확하게 요약해야해"
response3 = chain.invoke(question3)
print(f"\nQuestion3:\n{question3}\n",
      f"\nResponse3:\n{response3}")

question4 = "주인공이 멋진 이유가 뭘까?"
response4 = chain.invoke(question4)
print(f"\nQuestion4:\n{question4}\n",
      f"\nResponse4:\n{response4}")

hard chunk ver. small embedding model
chunk size: 2000
overlap size: 300
number of chunks: 474
number saved in vecstoredb: 474

Question1:
주요 등장인물들을 알려줘
 
Response1:
주요 등장인물들은 다음과 같습니다.

- Huck (Huckleberry Finn): 이야기의 주인공.
- Jim: Huck과 함께 여행하는 도망 노예.
- The King과 The Duke: 사기꾼 두 명으로, Huck과 Jim과 함께 다니며 여러 사기를 벌임.
- Buck: Huck이 만나는 소년.
- Bob과 Tom: Buck의 형들.
- Miss Charlotte: Buck의 누나, 25세.
- Miss Sophia: Buck의 또 다른 누나, 20세.
- The old gentleman: Buck 가족의 가장, 여러 농장과 많은 노예를 소유함.

이 외에도 다양한 인물들이 등장하지만, 위 인물들이 주요 등장인물입니다.

Question2:
한문단으로 내용요약을 해줘. 책을 읽어보지않은사람에게 흥미를 일으킬수있도록 아주 흥미진진하게 내용을 소개하는 느낌으로 써줘
 
Response2:
『The Adventures of Huckleberry Finn』은 자유분방한 소년 Huck과 도망 노예 Jim이 미시시피 강을 따라 뗏목을 타고 모험을 떠나는 이야기입니다. 이들은 각양각색의 인물들과 기상천외한 사건들을 겪으며, 속임수꾼들과의 아슬아슬한 만남, 시골 마을의 독특한 풍경, 그리고 인간의 본성과 자유에 대한 깊은 질문에 맞닥뜨립니다. Mark Twain 특유의 유머와 생생한 방언, 그리고 미국 남부의 진한 분위기가 어우러져, 한 번 읽기 시작하면 멈출 수 없는 흥미진진한 모험담이 펼쳐집니다.

Question3:
세문단으로 내용요약을 해줘. 기술적으로 사실에 근거해서 책의 모든 내용을 정확하게 요약해야해
 
Response3:
『The Adventures of Huc

In [34]:
question3 = "summarize the story by each chapter"
response3 = chain.invoke(question3)
print(f"\nQuestion3:\n{question3}\n",
      f"\nResponse3:\n{response3}")


Question3:
summarize the story by each chapter
 
Response3:
제공된 문맥에는 『The Adventures of Huckleberry Finn』의 각 장별 제목이나 구체적인 줄거리는 포함되어 있지 않습니다. 다만, 목차를 통해 각 장의 순서와 페이지 수만 확인할 수 있습니다. 따라서 각 장의 내용을 요약해서 설명드릴 수는 없습니다.

잘 모르겠습니다.


## chunk version. 2000, 400. embedding model = "text-embedding-3-large"

In [356]:

# Run Chain
print("hard chunk ver. large embedding model")
print(f"chunk size: {chunk_size_n}")
print(f"overlap size: {chunk_overlap_n}")
print(f"number of chunks: {len(split_documents)}")
print(f"number saved in vecstoredb: {vectorstore.index.ntotal}")


question1 = "주인공이 각각의 별에서 만난사람들을 나열해줘"
response1 = chain.invoke(question1)
print(f"\nQuestion1:\n{question1}\n", 
      f"\nResponse1:\n{response1}")

question2 = "주인공이 도착한 두 번째 별에선 누굴 만났고, 그사람한테 느낀 감정이 어땠어?"
response2 = chain.invoke(question2)
print(f"\nQuestion2:\n{question2}\n",
      f"\nResponse2:\n{response2}")

question3 = "주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?"
response3 = chain.invoke(question3)
print(f"\nQuestion3:\n{question3}\n",
      f"\nResponse3:\n{response3}")

question4 = "주인공이 두번째 별을 떠난 이유가 뭐라고 생각해?"
response4 = chain.invoke(question4)
print(f"\nQuestion4:\n{question4}\n",
      f"\nResponse4:\n{response4}")

hard chunk ver. large embedding model
chunk size: 2000
overlap size: 400
number of chunks: 43
number saved in vecstoredb: 43

Question1:
주인공이 각각의 별에서 만난사람들을 나열해줘
 
Response1:
주인공인 어린 왕자가 각각의 별에서 만난 사람들은 다음과 같습니다.

1. 첫 번째 별: 왕
2. 두 번째 별: 허영심 많은 사람(허영심쟁이)
3. 세 번째 별: 술꾼
4. 네 번째 별: 사업가(장사꾼)
5. 다섯 번째 별: 가로등을 켜는 사람(불 키는 사람)
6. 여섯 번째 별: 지리학자
7. 일곱 번째 별(지구): 여러 사람들을 만났지만, 처음에는 뱀을 만남

이렇게 어린 왕자는 각 별에서 다양한 어른들을 차례로 만나게 됩니다.

Question2:
주인공이 도착한 두 번째 별에선 누굴 만났고, 그사람한테 느낀 감정이 어땠어?
 
Response2:
주인공인 어린 왕자는 두 번째 별에서 허영심쟁이를 만났어. 허영심쟁이는 자신을 찬미해주길 바라는 사람이었지. 어린 왕자는 처음에는 허영심쟁이의 행동이 재미있다고 생각했지만, 곧 그 놀이가 무료해졌고, 허영심쟁이가 칭찬의 말 외에는 아무 말도 듣지 않는다는 사실에 실망했어. 그래서 결국 "어른들은 정말 별나다니깐."이라고 생각하며 그 별을 떠났어.

Question3:
주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?
 
Response3:
어린 왕자가 별들에서 만난 사람들 중에서 가장 싫어하는 인물은 장사꾼일 가능성이 높아 보입니다. 어린 왕자는 장사꾼과의 대화에서 별을 소유한다는 개념이나, 별을 단순히 숫자로 세고 종이에 적어두는 행위, 그리고 그것을 부의 상징으로 여기는 태도에 대해 전혀 만족스럽지 못해하고, 이해하지 못합니다. 어린 왕자는 장사꾼을 술꾼과 비슷하다고 속으로 생각하기도 하고, 어른들의 이런 사고방식이 괴상하다고 느낍니다. 또한, 장사꾼과의 대화가 끝난 뒤 곧바로 그곳

In [374]:

# Run Chain
print("hard chunk ver. large embedding model")
print(f"chunk size: {chunk_size_n}")
print(f"overlap size: {chunk_overlap_n}")
print(f"number of chunks: {len(split_documents)}")
print(f"number saved in vecstoredb: {vectorstore.index.ntotal}")


question1 = "주인공이 각각의 별에서 만난사람들을 나열해줘"
response1 = chain.invoke(question1)
print(f"\nQuestion1:\n{question1}\n", 
      f"\nResponse1:\n{response1}")

question2 = "주인공이 도착한 두 번째 별에선 누굴 만났고, 그사람한테 느낀 감정이 어땠어?"
response2 = chain.invoke(question2)
print(f"\nQuestion2:\n{question2}\n",
      f"\nResponse2:\n{response2}")

question3 = "주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?"
response3 = chain.invoke(question3)
print(f"\nQuestion3:\n{question3}\n",
      f"\nResponse3:\n{response3}")

question4 = "주인공이 두번째 별을 떠난 이유가 뭐라고 생각해?"
response4 = chain.invoke(question4)
print(f"\nQuestion4:\n{question4}\n",
      f"\nResponse4:\n{response4}")

hard chunk ver. large embedding model
chunk size: 2000
overlap size: 200
number of chunks: 43
number saved in vecstoredb: 43

Question1:
주인공이 각각의 별에서 만난사람들을 나열해줘
 
Response1:
주인공인 어린 왕자가 각각의 별에서 만난 사람들은 다음과 같습니다.

1. 첫 번째 별: 왕
2. 두 번째 별: 허영심쟁이
3. 세 번째 별: 술꾼
4. 네 번째 별: 사업가(장사꾼)
5. 다섯 번째 별: 가로등을 켜는 사람
6. 여섯 번째 별: 지리학자
7. 일곱 번째 별(지구): 여러 사람(뱀, 여우 등)과 만남

이렇게 어린 왕자는 각 별에서 다양한 어른들을 만나게 됩니다.

Question2:
주인공이 도착한 두 번째 별에선 누굴 만났고, 그사람한테 느낀 감정이 어땠어?
 
Response2:
주인공인 어린 왕자는 두 번째 별에서 허영심쟁이를 만났어. 허영심쟁이는 자신을 칭찬해주고 찬미해주길 바라는 사람이었지. 어린 왕자는 처음에는 허영심쟁이의 행동이 재미있다고 생각했지만, 곧 그 놀이가 무료해졌고, 허영심쟁이가 칭찬 말 외에는 아무 말도 듣지 않는다는 사실에 실망했어. 그래서 결국 "어른들은 정말 별나다니깐."이라고 생각하며 그 별을 떠났어. 어린 왕자는 허영심쟁이에 대해 이해하지 못하고, 어른들이 이상하다고 느꼈던 거야.

Question3:
주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?
 
Response3:
어린 왕자가 별들에서 만난 사람들 중에서 가장 싫어하는 인물은 명확하게 "누구"라고 직접적으로 말하지는 않지만, 대화와 묘사를 통해 유추해볼 수 있습니다. 어린 왕자는 장사꾼, 왕, 허영심쟁이, 술꾼 등 어른들의 행동과 사고방식을 이해하지 못하고, 그들을 "괴상하다"거나 "우스꽝스럽다"고 생각합니다. 특히 장사꾼과의 대화에서는 별을 소유한다는 개념 자체를 이해하지 못하고, 장사꾼을 술꾼과 비슷하다고 속으로 생각하며

In [370]:

question4 = "어린왕자는 다섯번째별을 떠난 이유는 사람이 이해가 가지않아서가 아니고 공간의 제약때문 아니야?"
response4 = chain.invoke(question4)
print(f"\nQuestion4:\n{question4}\n",
      f"\nResponse4:\n{response4}")


Question4:
어린왕자는 다섯번째별을 떠난 이유는 사람이 이해가 가지않아서가 아니고 공간의 제약때문 아니야?
 
Response4:
네, 맞습니다. 어린 왕자가 다섯 번째 별(불 키는 사람이 있는 별)을 떠난 주된 이유는 공간의 제약 때문입니다. 문서에 따르면, 어린 왕자는 "그의 별은 정말 너무도 작아. 두 사람이 있을 공간도 없어..."라고 말하며, 이별을 인정할 수밖에 없었다고 나와 있습니다. 즉, 불 키는 사람과 함께 있고 싶었지만, 별이 너무 작아서 두 사람이 함께 있을 수 없었기 때문에 떠나게 된 것입니다.


In [371]:

question4 = "아까는 불키는 사람이 이해가 안가서 떠난거라며?"
response4 = chain.invoke(question4)
print(f"\nQuestion4:\n{question4}\n",
      f"\nResponse4:\n{response4}")


Question4:
아까는 불키는 사람이 이해가 안가서 떠난거라며?
 
Response4:
아니요, 불 키는 사람을 떠난 이유는 그가 이해가 안 가서가 아니라, 불 키는 사람이 자신의 일에 성실하게 임하는 모습이 사랑스럽고, 그를 도와주고 싶었지만 결국 도움이 되지 못했기 때문입니다. 어린 왕자는 불 키는 사람을 우스꽝스럽게 여기지 않았고, 오히려 그를 좋아했습니다. 따라서 이해가 안 가서 떠난 것이 아니라, 더 이상 머물 이유가 없었기 때문에 떠난 것입니다.


## chunk version. 2000, 400. embedding model = "text-embedding-3-small"

In [ ]:

# Run Chain
print("## chunk version. 2000, 400. embedding model = "text-embedding-3-small"")
print(f"chunk size: {chunk_size_n}")
print(f"overlap size: {chunk_overlap_n}")
print(f"number of chunks: {len(split_documents)}")
print(f"number saved in vecstoredb: {vectorstore.index.ntotal}")


question1 = "주인공이 각각의 별에서 만난사람들을 나열해줘"
response1 = chain.invoke(question1)
print(f"\nQuestion1:\n{question1}\n", 
      f"\nResponse1:\n{response1}")

question2 = "주인공이 도착한 두 번째 별에선 누굴 만났고, 그사람한테 느낀 감정이 어땠어?"
response2 = chain.invoke(question2)
print(f"\nQuestion2:\n{question2}\n",
      f"\nResponse2:\n{response2}")

question3 = "주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?"
response3 = chain.invoke(question3)
print(f"\nQuestion3:\n{question3}\n",
      f"\nResponse3:\n{response3}")

question4 = "주인공이 두번째 별을 떠난 이유가 뭐라고 생각해?"
response4 = chain.invoke(question4)
print(f"\nQuestion4:\n{question4}\n",
      f"\nResponse4:\n{response4}")

## chunk version. 500, 100. embedding model = "text-embedding-3-small"

In [335]:

# Run Chain
print("hard chunk ver")
print(f"chunk size: {chunk_size_n}")
print(f"overlap size: {chunk_overlap_n}")
print(f"number of chunks: {len(split_documents)}")
print(f"number saved in vecstoredb: {vectorstore.index.ntotal}")


question1 = "주인공이 각각의 별에서 만난사람들을 나열해줘"
response1 = chain.invoke(question1)
print(f"\nQuestion1:\n{question1}\n", 
      f"\nResponse1:\n{response1}")

question2 = "주인공이 도착한 두 번째 별에선 누굴 만났고, 그사람한테 느낀 감정이 어땠어?"
response2 = chain.invoke(question2)
print(f"\nQuestion2:\n{question2}\n",
      f"\nResponse2:\n{response2}")

question3 = "주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?"
response3 = chain.invoke(question3)
print(f"\nQuestion3:\n{question3}\n",
      f"\nResponse3:\n{response3}")

question4 = "주인공이 두번째 별을 떠난 이유가 뭐라고 생각해?"
response4 = chain.invoke(question4)
print(f"\nQuestion4:\n{question4}\n",
      f"\nResponse4:\n{response4}")

hard chunk ver
chunk size: 500
overlap size: 100
number of chunks: 132
number saved in vecstoredb: 132

Question1:
주인공이 각각의 별에서 만난사람들을 나열해줘
 
Response1:
문맥에 따르면, 주인공(어린 왕자)이 각각의 별에서 만난 사람들 중 일부는 다음과 같습니다.

- 왕: 별을 소유하지 않고 통치만 하는 인물
- 장사꾼(상인): 별을 소유하고 있다고 주장하며, 별을 통해 부자가 되려고 하는 인물
- 술꾼: 장사꾼과 비슷하다고 어린 왕자가 언급함

문맥에 직접적으로 언급된 인물은 위와 같으며, 다른 별에서 만난 인물에 대한 정보는 주어지지 않았습니다.

Question2:
주인공이 도착한 두 번째 별에선 누굴 만났고, 그사람한테 느낀 감정이 어땠어?
 
Response2:
주인공인 어린 왕자가 도착한 두 번째 별에선 왕을 만났어. 왕은 자주빛 옷과 가운을 입고 근엄하게 왕좌에 앉아 있었고, 어린 왕자를 보자마자 "신하가 왔구먼!"이라고 소리쳤지. 어린 왕자는 자신을 한 번도 본 적이 없으면서 다짜고짜 신하라고 부르는 왕을 보고 혼잣말로 "나를 한 번도 본 적이 없으면서 다짜고짜 신하라니 참?"이라고 말했어. 이로 보아 어린 왕자는 왕에 대해 어이없고 당황스러운 감정을 느꼈던 것 같아.

Question3:
주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?
 
Response3:
주어진 문맥을 보면, 어린 왕자가 별들에서 만난 여러 인물 중 "장사꾼"에 대해 비판적으로 생각하는 부분이 나옵니다. 장사꾼은 별들을 소유하려 하고, 그 이유가 단지 자신을 부자로 만들어주기 때문이라고 말합니다. 어린 왕자는 속으로 "술꾼과 비슷하시군"이라고 생각하며, 장사꾼의 행동을 이해하지 못하고 비판적으로 바라봅니다.

따라서 문맥상 어린 왕자가 별들에서 만난 사람 중 가장 싫어하는 인물은 "장사꾼"일 가능성이 높다고 생각합니다.

Question4:
주인공이 두

## text splitter semantic model = "text-embedding-3-small"

In [340]:

# Run Chain
print("semantic chunk ver")

question1 = "주인공이 각각의 별에서 만난사람들을 나열해줘"
response1 = chain.invoke(question1)
print(f"\nQuestion1:\n{question1}\n", 
      f"\nResponse1:\n{response1}")

question2 = "주인공이 도착한 두 번째 별에선 누굴 만났고, 그사람한테 느낀 감정이 어땠어?"
response2 = chain.invoke(question2)
print(f"\nQuestion2:\n{question2}\n",
      f"\nResponse2:\n{response2}")

question3 = "주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?"
response3 = chain.invoke(question3)
print(f"\nQuestion3:\n{question3}\n",
      f"\nResponse3:\n{response3}")

question4 = "주인공이 두번째 별을 떠난 이유가 뭐라고 생각해?"
response4 = chain.invoke(question4)
print(f"\nQuestion4:\n{question4}\n",
      f"\nResponse4:\n{response4}")

semantic chunk ver

Question1:
주인공이 각각의 별에서 만난사람들을 나열해줘
 
Response1:
문맥에 따르면, 어린 왕자가 방문한 별들과 그곳에서 만난 사람들은 다음과 같습니다.

1. 첫 번째 별: 왕  
2. (문맥상 두 번째, 세 번째, 네 번째, 다섯 번째 별에 대한 직접적인 언급은 없으나, 별을 방문하며 여러 인물을 만난 것으로 보입니다.)
3. (다섯 번째 별에 대한 언급은 없으나, 여섯 번째 별이 등장합니다.)
4. 여섯 번째 별: 지리학자  
5. (또한, 별을 소유하고 있다고 주장하는 장사꾼(사업가)도 만납니다.)

정리하면, 문맥에서 명확히 언급된 인물은  
- 첫 번째 별: 왕  
- (별 번호는 명확하지 않으나) 장사꾼(사업가)  
- 여섯 번째 별: 지리학자  

이렇게 나열할 수 있습니다.

Question2:
주인공이 도착한 두 번째 별에선 누굴 만났고, 그사람한테 느낀 감정이 어땠어?
 
Response2:
주어진 문맥에는 어린 왕자가 첫 번째 별에서 왕을 만난 내용까지만 나와 있고, 두 번째 별에서 누구를 만났는지에 대한 직접적인 언급은 없습니다. 따라서 주인공이 두 번째 별에서 만난 인물과 그에 대한 감정은 잘 모르겠습니다.

Question3:
주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?
 
Response3:
주어진 문맥을 보면, 어린 왕자가 별들에서 만난 여러 어른들 중에서 "장사꾼"에 대해 특히 만족스럽지 못하고, 그의 사고방식을 이해하지 못해 불만을 느끼는 장면이 나옵니다. 장사꾼은 별들을 소유하고 있다고 주장하며, 별들을 세고 종이에 적어두는 것에 집착합니다. 어린 왕자는 이런 장사꾼의 행동을 "뚱딴지같다"고 생각하고, "어른들은 정말 다들 괴상해"라고 속으로 말합니다.

이러한 대화와 어린 왕자의 반응을 볼 때, 어린 왕자가 별들에서 만난 사람들 중 가장 싫어하는 인물은 "장사꾼"일 가능성이 높다고 생각합니다. 그는 장사꾼의 진지하지만 쓸모없어 보이는 행동과 사고방

## splitter semantic model = "text-embedding-3-large"

In [344]:

# Run Chain
print("semantic chunk ver")

question1 = "주인공이 각각의 별에서 만난사람들을 나열해줘"
response1 = chain.invoke(question1)
print(f"\nQuestion1:\n{question1}\n", 
      f"\nResponse1:\n{response1}")

question2 = "주인공이 도착한 두 번째 별에선 누굴 만났고, 그사람한테 느낀 감정이 어땠어?"
response2 = chain.invoke(question2)
print(f"\nQuestion2:\n{question2}\n",
      f"\nResponse2:\n{response2}")

question3 = "주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?"
response3 = chain.invoke(question3)
print(f"\nQuestion3:\n{question3}\n",
      f"\nResponse3:\n{response3}")

question4 = "주인공이 두번째 별을 떠난 이유가 뭐라고 생각해?"
response4 = chain.invoke(question4)
print(f"\nQuestion4:\n{question4}\n",
      f"\nResponse4:\n{response4}")

semantic chunk ver

Question1:
주인공이 각각의 별에서 만난사람들을 나열해줘
 
Response1:
문맥에 따르면, 어린 왕자가 방문한 별들과 그곳에서 만난 사람들은 다음과 같습니다.

1. 어린 왕자의 별(소행성 B 612): 어린 왕자 자신이 살고 있음.
2. (별 번호는 명확히 나오지 않지만) 한 별에서는 "성실한 사람"을 만남. 어린 왕자는 그를 친구로 삼고 싶어 했으나, 별이 너무 작아 두 사람이 있을 공간이 없어서 떠나게 됨.
3. 여섯 번째 별: 엄청 큰 책을 쓰고 있는 노신사(즉, 지리학자)를 만남.
4. 일곱 번째 별(지구): 여러 종류의 사람들(111명의 왕, 7천 명의 지리학자, 90만 명의 장사꾼, 750만 명의 술꾼, 3억 1,100만 명의 허영심쟁이 등)을 만날 수 있는 곳임.  
5. 또 다른 별에서는 상인을 만남.

정리하면, 어린 왕자는 각 별에서 다음과 같은 사람들을 만났습니다:
- 성실한 사람
- 노신사(지리학자)
- 상인
- 지구에서는 왕, 지리학자, 장사꾼, 술꾼, 허영심쟁이 등 다양한 사람들

문맥에 등장하지 않은 다른 별의 인물들은 잘 모르겠습니다.

Question2:
주인공이 도착한 두 번째 별에선 누굴 만났고, 그사람한테 느낀 감정이 어땠어?
 
Response2:
주어진 문맥에는 어린 왕자가 두 번째 별에서 만난 인물에 대한 직접적인 설명이 나오지 않습니다. 다만, 어린 왕자가 첫 번째 별에서는 왕을 만났고, 여섯 번째 별에서는 책을 쓰는 노신사를 만났다는 내용이 있습니다. 두 번째 별에 대한 구체적인 인물과 어린 왕자의 감정은 문맥에 나타나지 않으므로, 잘 모르겠습니다.

Question3:
주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?
 
Response3:
주어진 문맥을 보면, 어린 왕자가 별들에서 만난 여러 인물 중 장사꾼(사업가)에 대해 만족스럽지 못한 대화를 나누고, 그의 행동에 대해 비판적인 시각을 보입니다. 장사꾼은 별들을 소유하려 하고, 별들을 세고 또

## semantic default model

In [342]:

# Run Chain
print("semantic chunk ver")

question1 = "주인공이 각각의 별에서 만난사람들을 나열해줘"
response1 = chain.invoke(question1)
print(f"\nQuestion1:\n{question1}\n", 
      f"\nResponse1:\n{response1}")

question2 = "주인공이 도착한 두 번째 별에선 누굴 만났고, 그사람한테 느낀 감정이 어땠어?"
response2 = chain.invoke(question2)
print(f"\nQuestion2:\n{question2}\n",
      f"\nResponse2:\n{response2}")

question3 = "주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?"
response3 = chain.invoke(question3)
print(f"\nQuestion3:\n{question3}\n",
      f"\nResponse3:\n{response3}")

question4 = "주인공이 두번째 별을 떠난 이유가 뭐라고 생각해?"
response4 = chain.invoke(question4)
print(f"\nQuestion4:\n{question4}\n",
      f"\nResponse4:\n{response4}")

semantic chunk ver

Question1:
주인공이 각각의 별에서 만난사람들을 나열해줘
 
Response1:
주어진 문맥을 바탕으로 어린 왕자가 각각의 별에서 만난 사람들을 나열하면 다음과 같습니다.

- 첫 번째 별: 왕
- 두 번째 별: 허영심 많은 사람(허영꾼)
- 세 번째 별: 술꾼
- 네 번째 별: 사업가(장사꾼)
- 다섯 번째 별: 가로등 켜는 사람(가로등지기)
- 여섯 번째 별: 엄청 큰 책을 쓰고 있는 노신사(지리학자)

문맥에 직접적으로 언급된 인물은 위와 같으며, 각 별에서 만난 사람들의 특징도 일부 유추할 수 있습니다.

Question2:
주인공이 도착한 두 번째 별에선 누굴 만났고, 그사람한테 느낀 감정이 어땠어?
 
Response2:
주어진 문맥에는 어린 왕자가 첫 번째 별에서 왕을 만났다는 내용까지만 나와 있고, 두 번째 별에서 누구를 만났는지와 그에 대한 어린 왕자의 감정에 대한 직접적인 언급은 없습니다. 따라서 잘 모르겠습니다.

Question3:
주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?
 
Response3:
주어진 문맥에서는 주인공(어린 왕자)이 별들에서 만난 사람들 중에서 가장 싫어하는 인물이 누구인지 직접적으로 언급하고 있지 않습니다. 하지만 문맥을 살펴보면, 어린 왕자가 별들을 소유하려고 하는 장사꾼(사업가)과의 대화에서 만족스럽지 못하다고 느끼고, 별들을 세고 또 세는 행위에 대해 이해하지 못하며 부정적인 반응을 보입니다. 장사꾼은 별들을 소유하고 장사하기 위해 별을 세는 사람으로, 어린 왕자가 중요하게 생각하는 가치(사랑, 우정, 의미 등)와는 거리가 먼 인물입니다.

따라서 문맥상으로 유추해보면, 어린 왕자가 별들에서 만난 사람들 중 가장 싫어하는 인물은 별들을 소유하려는 장사꾼(사업가)일 가능성이 높다고 생각합니다.

Question4:
주인공이 두번째 별을 떠난 이유가 뭐라고 생각해?
 
Response4:
문맥에 따르면, 어린 왕자는 두 번째 별(즉, 어떤 별에서 만

## text split semantic model = "text-embedding-3-large", embedding model = "text-embedding-3-large"

In [348]:

# Run Chain
print(" text split semantic model = text-embedding-3-large, embedding model = text-embedding-3-large")

question1 = "주인공이 각각의 별에서 만난사람들을 나열해줘"
response1 = chain.invoke(question1)
print(f"\nQuestion1:\n{question1}\n", 
      f"\nResponse1:\n{response1}")

question2 = "주인공이 도착한 두 번째 별에선 누굴 만났고, 그사람한테 느낀 감정이 어땠어?"
response2 = chain.invoke(question2)
print(f"\nQuestion2:\n{question2}\n",
      f"\nResponse2:\n{response2}")

question3 = "주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?"
response3 = chain.invoke(question3)
print(f"\nQuestion3:\n{question3}\n",
      f"\nResponse3:\n{response3}")

question4 = "주인공이 두번째 별을 떠난 이유가 뭐라고 생각해?"
response4 = chain.invoke(question4)
print(f"\nQuestion4:\n{question4}\n",
      f"\nResponse4:\n{response4}")

 text split semantic model = text-embedding-3-large embedding model = text-embedding-3-large

Question1:
주인공이 각각의 별에서 만난사람들을 나열해줘
 
Response1:
문맥에 따르면, 주인공(어린 왕자)이 각각의 별에서 만난 사람들은 다음과 같습니다.

1. 첫 번째 별: 왕
2. 두 번째 별: 허영심쟁이
3. 세 번째 별: 술꾼
4. 네 번째 별: 사업가(장사꾼)
5. 다섯 번째 별: 잘 모르겠습니다 (문맥에 정보가 없습니다)
6. 여섯 번째 별: 지리학자
7. 일곱 번째 별: 지구(여러 인물들이 있지만, 구체적으로 누구를 만났는지는 문맥에 나오지 않습니다)

이렇게 어린 왕자는 각 별에서 다양한 사람들을 만났습니다.

Question2:
주인공이 도착한 두 번째 별에선 누굴 만났고, 그사람한테 느낀 감정이 어땠어?
 
Response2:
주인공인 어린 왕자가 도착한 두 번째 별에는 허영심쟁이가 살고 있었어요. 문맥에 따르면, 두 번째 별에 사는 사람은 "허영심쟁이"라고만 짧게 언급되어 있고, 어린 왕자가 그 사람을 만났을 때 느낀 감정에 대한 직접적인 설명은 나오지 않습니다. 하지만 앞선 별(예: 왕, 장사꾼 등)에서 어린 왕자가 느꼈던 당황스러움이나 의문, 그리고 상대방의 행동을 보고 약간은 어이없어하거나 이해하지 못하는 태도를 보였던 점을 미루어 볼 때, 허영심쟁이를 만났을 때도 비슷하게 당황하거나 이해하지 못하는 감정을 느꼈을 것으로 유추할 수 있습니다. 

정리하자면, 두 번째 별에서는 허영심쟁이를 만났고, 어린 왕자는 아마도 그 사람의 허영심 많은 행동을 보고 당황하거나 이해하지 못했을 것이라고 생각됩니다.

Question3:
주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?
 
Response3:
주어진 문맥을 보면, 어린 왕자가 별들에서 만난 사람들 중 장사꾼(사업가)에 대해 비판적인 시각을 가지고 있는 것을 알 수 있습니다. 장사꾼이 별

In [ ]:
print (f"""
       500이상 문장 갯수: {
           len(
               [i for i in split_documents if len(i.page_content) > 500]
           )
       }""")


for doc in vectorstore.similarity_search("두번째별", k=3):
    print(doc.page_content, '\nend!!!!!!')



주인공인 어린 왕자는 두 번째 별에서 허영심쟁이를 만났을 때 처음에는 그가 왕보다 더 재밌다고 생각해서 손뼉을 쳐주며 잠시 즐거워했지만, 곧 그 놀이가 무료해졌고, 허영심쟁이의 행동이 이해되지 않아 혼란스러워했어. 결국 어린 왕자는 "어른들은 정말 별나다니깐."이라고 혼잣말을 하며 그곳을 떠났지. 즉, 처음엔 약간의 흥미와 호기심이 있었지만, 곧 지루함과 어른들에 대한 이상함, 실망 같은 감정을 느꼈다고 할 수 있어.


In [294]:
# 단계 5: 검색기(Retriever) 생성. 고급

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4.1-2025-04-14", temperature=0)


retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

In [ ]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)


INFO:langchain.retrievers.multi_query:Generated queries: ['주인공이 별들에서 만난 사람들 중에서 가장 부정적으로 평가한 인물은 누구인가요?  ', '어린 왕자가 여러 별을 여행하며 만난 인물들 중에서 주인공이 가장 싫어한 사람은 누구일까요?  ', '주인공이 별을 여행하면서 만난 인물들 중에서 가장 반감을 느낀 대상은 누구라고 할 수 있나요?']


In [219]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever_from_llm, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:

# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?"
response = chain.invoke(question)
# print(response)
# 
# 

In [221]:

response

'주어진 문맥을 보면, 어린 왕자가 별들에서 만난 여러 인물 중에서 "장사꾼"에 대해 특히 만족스럽지 못한 대화를 나누고, 그를 "술꾼과 비슷하시군"이라고 속으로 말하는 등 부정적인 감정을 드러냅니다. 장사꾼은 별들을 소유하려 하고, 그 이유도 단지 부자가 되기 위해서라고 말합니다. 어린 왕자는 이런 태도에 대해 이해하지 못하고, 별들을 소유한다는 개념 자체에 회의적입니다.\n\n따라서 문맥상 어린 왕자가 별들에서 만난 사람들 중 가장 싫어하는 인물은 "장사꾼"이라고 생각할 수 있습니다.'

## 전체 코드

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/le_Petit_Prince.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

ValueError: File path data/SPRI_AI_Brief_2023년12월호_F.pdf is not a valid file or url

In [ ]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "삼성전자가 자체 개발한 AI 의 이름은?"
response = chain.invoke(question)
print(response)